In [1]:
import pandas as pd

In [2]:
def fat_print(value):
    return "\033[1m" + value +  "\033[0m"

In [3]:
import pandas as pd
import zipfile
import io
import requests

# URL of the outer ZIP file
url = 'https://archive.ics.uci.edu/static/public/222/bank+marketing.zip'

# Download the outer ZIP file
response = requests.get(url)
outer_zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# List all files in the outer ZIP
print("Files in the outer ZIP:", outer_zip_file.namelist())

# Choose the specific inner ZIP file you want to extract
inner_zip_filename = 'bank.zip'  # or 'bank.zip'

# Extract the inner ZIP file
with outer_zip_file.open(inner_zip_filename) as inner_zip:
    inner_zip_file = zipfile.ZipFile(io.BytesIO(inner_zip.read()))

    # List all files in the inner ZIP
    print("Files in the inner ZIP:", inner_zip_file.namelist())

    # Choose the specific CSV file you want to read
    csv_filename = 'bank-full.csv'  # Replace with the actual CSV file name

    # Read the specific CSV file
    with inner_zip_file.open(csv_filename) as file:
        df = pd.read_csv(file, sep=';')  # Use the correct separator if needed

# Now you can work with the DataFrame `df`
print(df.head())

Files in the outer ZIP: ['bank.zip', 'bank-additional.zip']
Files in the inner ZIP: ['bank-full.csv', 'bank-names.txt', 'bank.csv']
   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1     

In [4]:
column_list = ['age', 'job', 'marital', 'education', 'housing', 'balance','contact','day','month','duration','campaign','pdays','previous','poutcome','y']
df_hw03 = df[column_list]

In [5]:
df_hw03

,age,job,marital,education,housing,balance,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,yes,2143,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,yes,29,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,yes,2,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,yes,1506,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,telephone,17,nov,508,4,-1,0,unknown,no


In [6]:
#Q1 What is the most frequent observation (mode) for the column `education`
print(f"A1: The most frequent observation (mode) for the column education is {fat_print(df_hw03.education.value_counts().index[0])}")

A1: The most frequent observation (mode) for the column education is secondary


In [7]:
#Create a correlation matrix for the numerical features
#Q2 What are the two features tha have the biggest correlation
numerical = df_hw03.select_dtypes(include=['number']).columns.to_list()
df_numeric = df_hw03[numerical]


In [8]:
sorted_correlations = df_numeric.corr().abs().unstack().sort_values(ascending=False)
highest_correlation_pair = sorted_correlations[sorted_correlations < 1].idxmax()
highest_correlation_value = sorted_correlations[highest_correlation_pair]

print(f"A2 The two features that have highest correlation value of {round(highest_correlation_value,2)} are {fat_print(highest_correlation_pair[0])} and {fat_print(highest_correlation_pair[1])} ")

A2 The two features that have highest correlation value of 0.45 are pdays and previous 


In [9]:
#Q3
df_hw03.loc[:, 'y'] = (df_hw03.y == 'yes').astype('int')

In [10]:
from sklearn.model_selection import train_test_split

df_train_full, df_test = train_test_split(df_hw03, test_size=0.2, random_state=42)
df_train, df_val =train_test_split(df_train_full, test_size=0.25, random_state=42)

In [11]:
from sklearn.metrics import mutual_info_score
mutual_info = {}
for col in ['contact', 'education', 'housing', 'poutcome']:
    mutual_info[col] = round(mutual_info_score(df_train_full.y, df_train_full[col]),2)

max_mi = max(mutual_info, key=mutual_info.get)
max_value = mutual_info[max_mi]
print(f"A3 The variable {fat_print(max_mi)} has the highest mutual information score with {max_value}")

A3 The variable poutcome has the highest mutual information score with 0.03


In [12]:
from sklearn.feature_extraction import DictVectorizer
categorical = df_hw03.select_dtypes(include=['object', 'category']).drop('y', axis=1).columns.to_list()

In [13]:
train_dicts = df_train[numerical + categorical].to_dict(orient='records')

In [14]:
dv = DictVectorizer(sparse=False)

In [15]:
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_train = df_train['y'].astype('int')
y_val = df_val['y'].astype('int')


In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [18]:
model.fit(X_train, y_train.astype('int'))


LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [19]:
client_subscription =  (model.predict_proba(X_val)[:,1])>0.5
print(f"A4 The accuracy I got is {round((y_val == client_subscription).mean(),2)}")

A4 The accuracy I got is 0.9


In [37]:
from sklearn.metrics import accuracy_score

def get_accuracy(feature_vector, C=1.0):
    # Transform the training and validation data using the selected features
    X_train_selected = df_train[feature_vector].to_dict(orient='records')
    X_val_selected = df_val[feature_vector].to_dict(orient='records')
    
    # Vectorize the selected features
    dv_small = DictVectorizer(sparse=False)
    X_train_transformed = dv_small.fit_transform(X_train_selected)
    X_val_transformed = dv_small.transform(X_val_selected)
    
    # Train the model
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train_transformed, y_train)
    print("N_iteration: :", model.n_iter_)
    
    # Predict on the validation set
    y_pred = model.predict(X_val_transformed)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy with features {feature_vector}: {accuracy}")
    return accuracy

In [22]:
feature_list = numerical + categorical

In [40]:
# Calculate baseline accuracy with all features
baseline_accuracy = get_accuracy(feature_list)

accuracy_differences = {}
for var in feature_list:
    reduced_feature_list = [f for f in feature_list if f != var]
    accuracy = get_accuracy(reduced_feature_list)
    accuracy_differences[var] = round(abs(baseline_accuracy - accuracy),2)



N_iteration: : [47]
Accuracy with features ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.9010174740101747
N_iteration: : [37]
Accuracy with features ['balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.9012386640123866
N_iteration: : [26]
Accuracy with features ['age', 'day', 'duration', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.9011280690112807
N_iteration: : [35]
Accuracy with features ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.900464499004645
N_iteration: : [34]
Accuracy with features ['age', 'balance', 'day', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.889515593895155

In [41]:
sorted_features = sorted(accuracy_differences.items(), key=lambda x: x[1])

# Print the ranking of features
print("Feature ranking based on accuracy difference:")
for feature, diff in sorted_features:
    print(f"{feature}: {diff:.4f}")

Feature ranking based on accuracy difference:
age: 0.0000
balance: 0.0000
day: 0.0000
campaign: 0.0000
pdays: 0.0000
previous: 0.0000
job: 0.0000
marital: 0.0000
education: 0.0000
housing: 0.0000
contact: 0.0000
month: 0.0000
duration: 0.0100
poutcome: 0.0100


In [47]:
print(f"A5: The variable with the lowest distance is {fat_print(sorted_features[0][0])}")

A5: The variable with the lowest distance is age


In [48]:
#Q6 Train regularized Logistic Regression which of the paramter for C
#[0.01, 0.1, 1, 10, 100] gives the best accuracy on 3 decimal digits
C_list  = [0.01, 0.1, 1, 10, 100]
for C in C_list:
    print(f"C = {C}")
    print(f"Accuracy: {round(get_accuracy(feature_vector=feature_list, C=C),3)}")
    

C = 0.01
N_iteration: : [35]
Accuracy with features ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.8980314089803141
Accuracy: 0.898
C = 0.1
N_iteration: : [41]
Accuracy with features ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.9009068790090687
Accuracy: 0.901
C = 1
N_iteration: : [44]
Accuracy with features ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.900353904003539
Accuracy: 0.9
C = 10
N_iteration: : [47]
Accuracy with features ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']: 0.9011280690112807
Accuracy: 0.901
C = 100
N_iteration: : [47]
Accuracy with features ['age', 'balance', 'day', 